<a href="https://colab.research.google.com/github/isshii/de4test/blob/add_deepxpore/deepxprore/deepXplore_for_kaggle_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# 事前準備
# 以下のディレクトリを Google Drive に作成してください
# /content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore
# /content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet
# /content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet/generated_inputs -> 生成した画像の保管場所
# /content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet/seeds -> seed となる画像の保管場所
# /content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/models -> 参照先のモデルの格納場所

from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#import argparse

%tensorflow_version 1.x

# from keras.applications.mobilenet import MobileNet
# from keras.applications.densenet import DenseNet201
# from keras.applications.nasnet import NASNetMobile


from keras.layers import Input
#from scipy.misc import imsave  
import imageio

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

#from utils import *


In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16251251418831802702, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13151239860993879178
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 2377359575964702272
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 421437817384018183
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
# read the parameter
# argument parsing
parser = argparse.ArgumentParser(description='Main function for difference-inducing input generation in MNIST dataset')
parser.add_argument('transformation', help="realistic transformation type", choices=['light', 'occl', 'blackout'])
parser.add_argument('weight_diff', help="weight hyperparm to control differential behavior", type=float)
parser.add_argument('weight_nc', help="weight hyperparm to control neuron coverage", type=float)
parser.add_argument('step', help="step size of gradient descent", type=float)
parser.add_argument('seeds', help="number of seeds of input", type=int)
parser.add_argument('grad_iterations', help="number of iterations of gradient descent", type=int)
parser.add_argument('threshold', help="threshold for determining neuron activated", type=float)
parser.add_argument('-t', '--target_model', help="target model that we want it predicts differently",
                    choices=[0, 1, 2], default=0, type=int)
parser.add_argument('-sp', '--start_point', help="occlusion upper left corner coordinate", default=(0, 0), type=tuple)
parser.add_argument('-occl_size', '--occlusion_size', help="occlusion size", default=(10, 10), type=tuple)

#args = parser.parse_args([])
args = parser.parse_args(["occl", "0.1", "0.1", "1", "100", "100", "0.1"])

In [0]:
import random
from collections import defaultdict

import numpy as np
from keras import backend as K
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import Model
from keras.preprocessing import image
import os, re


def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    input_img_data = image.img_to_array(img)
    input_img_data = np.expand_dims(input_img_data, axis=0)
    input_img_data = preprocess_input(input_img_data)  # final input shape = (1,224,224,3)
    return input_img_data


def deprocess_image(x):
    x = x.reshape((224, 224, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x


def decode_label(pred):
    return decode_predictions(pred)[0][0][1]


def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)


def constraint_occl(gradients, start_point, rect_shape):
    new_grads = np.zeros_like(gradients)
    new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
    start_point[1]:start_point[1] + rect_shape[1]] = gradients[:, start_point[0]:start_point[0] + rect_shape[0],
                                                     start_point[1]:start_point[1] + rect_shape[1]]
    return new_grads


def constraint_light(gradients):
    new_grads = np.ones_like(gradients)
    grad_mean = 1e4 * np.mean(gradients)
    return grad_mean * new_grads


def constraint_black(gradients, rect_shape=(10, 10)):
    start_point = (
        random.randint(0, gradients.shape[1] - rect_shape[0]), random.randint(0, gradients.shape[2] - rect_shape[1]))
    new_grads = np.zeros_like(gradients)
    patch = gradients[:, start_point[0]:start_point[0] + rect_shape[0], start_point[1]:start_point[1] + rect_shape[1]]
    if np.mean(patch) < 0:
        new_grads[:, start_point[0]:start_point[0] + rect_shape[0],
        start_point[1]:start_point[1] + rect_shape[1]] = -np.ones_like(patch)
    return new_grads


def init_coverage_tables(model1, model2, model3):
    model_layer_dict1 = defaultdict(bool)
    model_layer_dict2 = defaultdict(bool)
    model_layer_dict3 = defaultdict(bool)
    init_dict(model1, model_layer_dict1)
    init_dict(model2, model_layer_dict2)
    init_dict(model3, model_layer_dict3)
    return model_layer_dict1, model_layer_dict2, model_layer_dict3


def init_dict(model, model_layer_dict):
    for layer in model.layers:
        if 'flatten' in layer.name or 'input' in layer.name:
            continue
        for index in range(layer.output_shape[-1]):
            model_layer_dict[(layer.name, index)] = False


def neuron_to_cover(model_layer_dict):
    not_covered = [(layer_name, index) for (layer_name, index), v in list(model_layer_dict.items()) if not v]
    if not_covered:
        layer_name, index = random.choice(not_covered)
    else:
        layer_name, index = random.choice(list(model_layer_dict.keys()))
    return layer_name, index


def neuron_covered(model_layer_dict):
    covered_neurons = len([v for v in list(model_layer_dict.values()) if v])
    total_neurons = len(model_layer_dict)
    return covered_neurons, total_neurons, covered_neurons / float(total_neurons)


def scale(intermediate_layer_output, rmax=1, rmin=0):
    X_std = (intermediate_layer_output - intermediate_layer_output.min()) / (
        intermediate_layer_output.max() - intermediate_layer_output.min())
    X_scaled = X_std * (rmax - rmin) + rmin
    return X_scaled


def update_coverage(input_data, model, model_layer_dict, threshold=0):
    layer_names = [layer.name for layer in model.layers if
                   'flatten' not in layer.name and 'input' not in layer.name]

    intermediate_layer_model = Model(inputs=model.input,
                                     outputs=[model.get_layer(layer_name).output for layer_name in layer_names])
    intermediate_layer_outputs = intermediate_layer_model.predict(input_data)

    for i, intermediate_layer_output in enumerate(intermediate_layer_outputs):
        scaled = scale(intermediate_layer_output[0])
        for num_neuron in range(scaled.shape[-1]):
            if np.mean(scaled[..., num_neuron]) > threshold and not model_layer_dict[(layer_names[i], num_neuron)]:
                model_layer_dict[(layer_names[i], num_neuron)] = True


def full_coverage(model_layer_dict):
    if False in list(model_layer_dict.values()):
        return False
    return True


def fired(model, layer_name, index, input_data, threshold=0):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_layer_output = intermediate_layer_model.predict(input_data)[0]
    scaled = scale(intermediate_layer_output)
    if np.mean(scaled[..., index]) > threshold:
        return True
    return False


def diverged(predictions1, predictions2, predictions3, target):
    #     if predictions2 == predictions3 == target and predictions1 != target:
    if not predictions1 == predictions2 == predictions3:
        return True
    return False

#add
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [0]:
%%time
# input image dimensions
img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)
# model これでいける？
from keras import models
from keras.models import model_from_json
# define input tensor as a placeholder
input_tensor = Input(shape=input_shape)

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
vgg19_weights = '/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/models/vgg19_weights_tf_dim_ordering_tf_kernels.h5'
resnet50_weights = '/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/models/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
vgg16_weights = '/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5'
# model1 = VGG19(weights=vgg19_weights)
# model2 = VGG16(weights=vgg16_weights)
# model3 = ResNet50(weights=resnet50_weights)

model1 = VGG19(weights=vgg19_weights,input_tensor=input_tensor)
model2 = VGG16(weights=vgg16_weights,input_tensor=input_tensor)
model3 = ResNet50(weights=resnet50_weights,input_tensor=input_tensor)

# load multiple models sharing same input tensor
K.set_learning_phase(0)
# model1_1 = VGG19(input_tensor=input_tensor)
# model2_1 = ResNet50(input_tensor=input_tensor)
# model3_1 = VGG16(input_stensor=input_tensor)

# model1 = model1_1.load_weights('/content/gdrive/My Drive/deepxplore/models/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
# model2 = model2_1.load_weights('/content/gdrive/My Drive/deepxplore/models/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
# model3 = model3_1.load_weights('/content/gdrive/My Drive/deepxplore/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5')


# MODEL_ARC_PATH = '/content/gdrive/My Drive/deepxplore/models/vgg19_weights_tf_dim_ordering_tf_kernels.h5'

# model_arc_str = open(MODEL_ARC_PATH,encoding="utf8").read()
# model = model_from_json(model_arc_str)

# モデル構成の確認
# model.summary()

# model1 = models.load_model('/content/gdrive/My Drive/deepxplore/models/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
# model2 = models.load_model('/content/gdrive/My Drive/deepxplore/models/resnet50_weights_tf_dim_ordering_tf_kernels.h5')
# model3 = models.load_model('/content/gdrive/My Drive/deepxplore/models/vgg16_weights_tf_dim_ordering_tf_kernels.h5')

# init coverage table
model_layer_dict1, model_layer_dict2, model_layer_dict3 = init_coverage_tables(model1, model2, model3)

CPU times: user 15 s, sys: 908 ms, total: 15.9 s
Wall time: 17.3 s


In [0]:
# start gen inputs
#img_paths = image.list_pictures('./seeds', ext='JPEG')
img_paths = list_pictures('/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet/seeds')
gen_img = preprocess_image(random.choice(img_paths))
# print(gen_img)


In [0]:
# print(args.seeds)
# from keras.utils import plot_model
# plot_model(model1, to_file='model1.png',show_shapes=True)

In [0]:
# plot_model(model3, to_file='model3.png',show_shapes=True)

In [0]:
# t_model(model2, to_file='model2.png',show_shapes=True)

In [0]:
%%time
for _ in range(args.seeds):
    gen_img = preprocess_image(random.choice(img_paths))
    orig_img = gen_img.copy()
    # first check if input already induces differences
    pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
    label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])
    
    if not label1 == label2 == label3:
      ######################## 摂動を加えなくても予測が3つのモデルで異なる場合の処理 Start ########################
#        print(bcolors.OKGREEN + 'input already causes different outputs: {}, {}, {}'.format(decode_label(pred1),
        print('input already causes different outputs: {}, {}, {}'.format(decode_label(pred1),
                                                                                            decode_label(pred2),
                                                                                            decode_label(
                                                                                                pred3)) + bcolors.ENDC)
#                                                                                                pred3)))

        update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
        update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
        update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

        print(bcolors.OKGREEN + 'covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
#        print('covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
              % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
                 neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
                 neuron_covered(model_layer_dict3)[2]) + bcolors.ENDC)
#                 neuron_covered(model_layer_dict3)[2]))
  
        averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                       neuron_covered(model_layer_dict3)[0]) / float(
            neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
            neuron_covered(model_layer_dict3)[
                1])
        print(bcolors.OKGREEN + 'averaged covered neurons %.3f' % averaged_nc + bcolors.ENDC)
        print('averaged covered neurons %.3f' % averaged_nc)

        gen_img_deprocessed = deprocess_image(gen_img)

        # save the result to disk
        #imsave('./generated_inputs/' + 'already_differ_' + decode_label(pred1) + '_' + decode_label(
        imageio.imwrite('/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet/generated_inputs/' + 'already_differ_' + decode_label(pred1) + '_' + decode_label(
            pred2) + '_' + decode_label(pred3) + '.png',
               gen_img_deprocessed)
        continue
        ######################## 摂動を加えなくても予測が3つのモデルで異なる場合の処理END ########################

    ######################## 予測が3つのモデルで同じ場合の処理 Start ########################
    # if all label agrees
    orig_label = label1
    layer_name1, index1 = neuron_to_cover(model_layer_dict1)
    layer_name2, index2 = neuron_to_cover(model_layer_dict2)
    layer_name3, index3 = neuron_to_cover(model_layer_dict3)
    # construct joint loss function
    if args.target_model == 0:
        loss1 = -args.weight_diff * K.mean(model1.get_layer('predictions').output[..., orig_label])
        loss2 = K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('fc1000').output[..., orig_label])
    elif args.target_model == 1:
        loss1 = K.mean(model1.get_layer('predictions').output[..., orig_label])
        loss2 = -args.weight_diff * K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = K.mean(model3.get_layer('fc1000').output[..., orig_label])
    elif args.target_model == 2:
        loss1 = K.mean(model1.get_layer('predictions').output[..., label1])
        loss2 = K.mean(model2.get_layer('predictions').output[..., orig_label])
        loss3 = -args.weight_diff * K.mean(model3.get_layer('fc1000').output[..., orig_label])
    loss1_neuron = K.mean(model1.get_layer(layer_name1).output[..., index1])
    loss2_neuron = K.mean(model2.get_layer(layer_name2).output[..., index2])
    loss3_neuron = K.mean(model3.get_layer(layer_name3).output[..., index3])
    layer_output = (loss1 + loss2 + loss3) + args.weight_nc * (loss1_neuron + loss2_neuron + loss3_neuron)    

    # for adversarial image generation
    final_loss = K.mean(layer_output)

    # we compute the gradient of the input picture wrt this loss
    grads = normalize(K.gradients(final_loss, input_tensor)[0])#####  エラー箇所
    ##本来は K.gradients(final_loss, input_tensor)[0] = Tensor("gradients_8/AddN_18:0", shape=(?, 224, 224, 3), dtype=float32) みたいな値を返すっぽい

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_tensor], [loss1, loss2, loss3, loss1_neuron, loss2_neuron, loss3_neuron, grads])

    # we run gradient ascent for 20 steps
    for iters in range(args.grad_iterations):
        loss_value1, loss_value2, loss_value3, loss_neuron1, loss_neuron2, loss_neuron3, grads_value = iterate(
            [gen_img])
        if args.transformation == 'light':
            grads_value = constraint_light(grads_value)  # constraint the gradients value
        elif args.transformation == 'occl':
            grads_value = constraint_occl(grads_value, args.start_point,
                                          args.occlusion_size)  # constraint the gradients value
        elif args.transformation == 'blackout':
            grads_value = constraint_black(grads_value)  # constraint the gradients value

        gen_img += grads_value * args.step
        pred1, pred2, pred3 = model1.predict(gen_img), model2.predict(gen_img), model3.predict(gen_img)
        label1, label2, label3 = np.argmax(pred1[0]), np.argmax(pred2[0]), np.argmax(pred3[0])

        if not label1 == label2 == label3:
            update_coverage(gen_img, model1, model_layer_dict1, args.threshold)
            update_coverage(gen_img, model2, model_layer_dict2, args.threshold)
            update_coverage(gen_img, model3, model_layer_dict3, args.threshold)

            print(bcolors.OKGREEN + 'covered neurons percentage %d neurons %.3f, %d neurons %.3f, %d neurons %.3f'
                  % (len(model_layer_dict1), neuron_covered(model_layer_dict1)[2], len(model_layer_dict2),
                     neuron_covered(model_layer_dict2)[2], len(model_layer_dict3),
                     neuron_covered(model_layer_dict3)[2]) + bcolors.ENDC)
            averaged_nc = (neuron_covered(model_layer_dict1)[0] + neuron_covered(model_layer_dict2)[0] +
                           neuron_covered(model_layer_dict3)[0]) / float(
                neuron_covered(model_layer_dict1)[1] + neuron_covered(model_layer_dict2)[1] +
                neuron_covered(model_layer_dict3)[
                    1])
            print(bcolors.OKGREEN + 'averaged covered neurons %.3f' % averaged_nc + bcolors.ENDC)
            print('averaged covered neurons %.3f' % averaged_nc)

            gen_img_deprocessed = deprocess_image(gen_img)
            orig_img_deprocessed = deprocess_image(orig_img)

            # save the result to disk
            #imsave('./generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
            imageio.imwrite('/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet/generated_inputs
            /' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
                pred2) + '_' + decode_label(pred3) + '.png', gen_img_deprocessed)
            #imsave('./generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
            imageio.imwrite('/content/gdrive/My Drive/ColabNotebooks/test4ai/deepxprore/ImageNet/generated_inputs/' + args.transformation + '_' + decode_label(pred1) + '_' + decode_label(
              pred2) + '_' + decode_label(pred3) + '_orig.png', orig_img_deprocessed)
            break

input already causes different outputs: television, entertainment_center, television
covered neurons percentage 16168 neurons 0.248, 14888 neurons 0.244, 94123 neurons 0.762
averaged covered neurons 0.634
averaged covered neurons 0.634
input already causes different outputs: boa_constrictor, diamondback, boa_constrictor
covered neurons percentage 16168 neurons 0.302, 14888 neurons 0.294, 94123 neurons 0.766
averaged covered neurons 0.650
averaged covered neurons 0.650
input already causes different outputs: gown, lampshade, hoopskirt
covered neurons percentage 16168 neurons 0.338, 14888 neurons 0.336, 94123 neurons 0.771
averaged covered neurons 0.663
averaged covered neurons 0.663
input already causes different outputs: water_tower, yurt, suspension_bridge
covered neurons percentage 16168 neurons 0.380, 14888 neurons 0.378, 94123 neurons 0.776
averaged covered neurons 0.678
averaged covered neurons 0.678
input already causes different outputs: magpie, vulture, magpie
covered neurons p

KeyboardInterrupt: ignored